# Long Sentence Resolution - Create Splitting Modeling Dataset

# Documentation

## Connect Google Drive
 
* Run the cell "Connect Google Drive" and on the popup window click "Connect to Google Drive".
 
* In the new page opened, choose an account (your account) and click Allow.

## Configuration parameters

* **Preprocessed data**: The preprocessed data that will be loaded for creating the modeling dataset.

* **Preprocessed data full paths**: The full paths to use from the selected preprocessed data.

* **Datasets to keep after splitting in train-validation splits**: The datasets to use from the selected preprocessed data for the train-validation splits after splitting.

* **Datasets to keep after splitting in test split**: The datasets to use from the selected preprocessed data for the test split after splitting.

* **Train split ratio**: The train split ratio to use when splitting the selected preprocessed data (default: 0.8).

* **Validation split ratio**: The validation split ratio to use when splitting the selected preprocessed data (default: 0.1).

* **Test split ratio**: The test split ratio to use when splitting the selected preprocessed data (default: 0.1).

* **Shuffle data when splitting?**: Whether the selected preprocessed data will be shuffled when splitting it (default: True).

* **RNG seed**: The seed that will be used for the random number generator when shuffling the selected preprocessed data (default: 42).

# Installations

In [ ]:
!pip install datasets==2.5.2

# Imports

In [ ]:
import os, datetime, json

from pathlib import Path

from typing import List

from datasets import load_from_disk, Dataset, DatasetDict

from google.colab import drive

import ipywidgets as widgets

# Connect Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Create paths

In [ ]:
PREPROCESSED_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/preprocessed')

MODELING_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/modeling')

MODELING_DATA_DIRECTORY_PATH.mkdir(parents=True, exist_ok=True)

# Functions / Classes

In [ ]:
def load_full_paths_from_preprocessed_data_path (data_path: Path) -> List[str]:
  with open(data_path / 'config.json') as f: return json.load(f) ['full_paths_in_preprocessed_data']

def extract_dataset_names_from_full_paths (full_paths: List[str]) -> List[str]:
  return list(sorted(set(map(lambda o: o.split('>') [0], full_paths))))

def split_dataset_in_train_validation_test (dataset: Dataset, train_split_ratio: float, validation_split_ratio: float, test_split_ratio: float, shuffle: bool, stratify_by_column: str, seed: int) -> DatasetDict:
  assert(train_split_ratio + validation_split_ratio + test_split_ratio == 1.0)

  trainvalidation_test = dataset.train_test_split(test_size=test_split_ratio, shuffle=shuffle, seed=seed, stratify_by_column=stratify_by_column)

  train_validation = trainvalidation_test['train'].train_test_split(test_size=validation_split_ratio / (train_split_ratio + validation_split_ratio), shuffle=shuffle, seed=seed, stratify_by_column=stratify_by_column)

  return DatasetDict({
    'train': train_validation['train'],
    'validation': train_validation['test'],
    'test': trainvalidation_test['test']
  })

# Configuration parameters

In [ ]:
def on_preprocessed_data_dropdown_change(change):
  preprocessed_data_full_paths_select_multiple.options = preprocessed_data_full_paths_select_multiple.value = load_full_paths_from_preprocessed_data_path(PREPROCESSED_DATA_DIRECTORY_PATH / change.new)

def on_preprocessed_data_full_paths_select_multiple_change(change):
  datasets_to_keep_after_splitting_in_test_split_select_multiple.options = \
  datasets_to_keep_after_splitting_in_test_split_select_multiple.value = \
  datasets_to_keep_after_splitting_in_train_validation_splits_select_multiple.options = \
  datasets_to_keep_after_splitting_in_train_validation_splits_select_multiple.value = \
  extract_dataset_names_from_full_paths(change.new)

preprocessed_data_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('splitting_dataset'), os.listdir(PREPROCESSED_DATA_DIRECTORY_PATH))))

preprocessed_data_full_paths = load_full_paths_from_preprocessed_data_path(PREPROCESSED_DATA_DIRECTORY_PATH / preprocessed_data_dropdown.value)
preprocessed_data_full_paths_select_multiple = widgets.SelectMultiple(options=preprocessed_data_full_paths, value=preprocessed_data_full_paths, rows=10, layout={'width': '400px'})

dataset_names_from_preprocessed_data_full_paths = extract_dataset_names_from_full_paths(preprocessed_data_full_paths)
datasets_to_keep_after_splitting_in_train_validation_splits_select_multiple = widgets.SelectMultiple(options=dataset_names_from_preprocessed_data_full_paths, value=dataset_names_from_preprocessed_data_full_paths, rows=10, layout={'width': '400px'})
datasets_to_keep_after_splitting_in_test_split_select_multiple = widgets.SelectMultiple(options=dataset_names_from_preprocessed_data_full_paths, value=dataset_names_from_preprocessed_data_full_paths, rows=10, layout={'width': '400px'})

train_split_ratio_bounded_float_text = widgets.BoundedFloatText(value=0.87, min=0.0, max=1.0, step=0.01)
validation_split_ratio_bounded_float_text = widgets.BoundedFloatText(value=0.03, min=0.0, max=1.0, step=0.01)
test_split_ratio_bounded_float_text = widgets.BoundedFloatText(value=0.1, min=0.0, max=1.0, step=0.01)

rng_seed_bounded_int_text = widgets.BoundedIntText(value=42, min=1,  max=10000, step=1)
shuffle_data_when_splitting_checkbox = widgets.Checkbox(value=True, indent=False)

preprocessed_data_dropdown.observe(on_preprocessed_data_dropdown_change, names='value')
preprocessed_data_full_paths_select_multiple.observe(on_preprocessed_data_full_paths_select_multiple_change, names='value')

display(widgets.HBox([widgets.Label(value="Preprocessed data:"), preprocessed_data_dropdown]))
display(widgets.HBox([widgets.Label(value="Preprocessed data full paths:"), preprocessed_data_full_paths_select_multiple]))
display(widgets.HBox([widgets.Label(value="Datasets to keep after splitting in train-validation splits:"), datasets_to_keep_after_splitting_in_train_validation_splits_select_multiple]))
display(widgets.HBox([widgets.Label(value="Datasets to keep after splitting in test split:"), datasets_to_keep_after_splitting_in_test_split_select_multiple]))
display(widgets.HBox([widgets.Label(value="Train split ratio:"), train_split_ratio_bounded_float_text]))
display(widgets.HBox([widgets.Label(value="Validation split ratio:"), validation_split_ratio_bounded_float_text]))
display(widgets.HBox([widgets.Label(value="Test split ratio:"), test_split_ratio_bounded_float_text]))
display(widgets.HBox([widgets.Label(value="Shuffle data when splitting?"), shuffle_data_when_splitting_checkbox]))
display(widgets.HBox([widgets.Label(value="RNG Seed:"), rng_seed_bounded_int_text]))

# Load preprocessed data

In [ ]:
%%time

preprocessed_data = load_from_disk(PREPROCESSED_DATA_DIRECTORY_PATH / preprocessed_data_dropdown.value)

preprocessed_data

# Filter preprocessed data

In [ ]:
filtered_data = preprocessed_data.filter(lambda o: o['full_path'] in preprocessed_data_full_paths_select_multiple.value)

filtered_data

# Split filtered data into train-validation-test

In [ ]:
splitted_data = split_dataset_in_train_validation_test(filtered_data.map(lambda o: { 'dataset_as_label': o['dataset'] }).class_encode_column("dataset_as_label"),
                                                       train_split_ratio = train_split_ratio_bounded_float_text.value,
                                                       validation_split_ratio = validation_split_ratio_bounded_float_text.value,
                                                       test_split_ratio = test_split_ratio_bounded_float_text.value,
                                                       shuffle = shuffle_data_when_splitting_checkbox.value,
                                                       stratify_by_column='dataset_as_label',
                                                       seed = rng_seed_bounded_int_text.value)

splitted_data = splitted_data.remove_columns('dataset_as_label')

splitted_data

# Keep only the selected datasets from splitted data

In [ ]:
modeling_dataset = DatasetDict({
    "train": splitted_data['train'].filter(lambda o: o['dataset'] in datasets_to_keep_after_splitting_in_train_validation_splits_select_multiple.value),
    
    "validation": splitted_data['validation'].filter(lambda o: o['dataset'] in datasets_to_keep_after_splitting_in_train_validation_splits_select_multiple.value),
    
    "test": splitted_data['test'].filter(lambda o: o['dataset'] in datasets_to_keep_after_splitting_in_test_split_select_multiple.value)
})

modeling_dataset

# Save modeling dataset

In [ ]:
modeling_dataset_directory_path = MODELING_DATA_DIRECTORY_PATH / f'splitting_dataset_{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}'

os.makedirs(modeling_dataset_directory_path)

In [ ]:
for split, data in modeling_dataset.items():
  data.to_csv(modeling_dataset_directory_path / f"{split}.csv", index=None, na_rep='None')

with open(PREPROCESSED_DATA_DIRECTORY_PATH / preprocessed_data_dropdown.value / 'config.json', 'r') as input_file:
  with open(modeling_dataset_directory_path / 'config.json', 'w') as output_file: json.dump({
    "selected_preprocessed_data" : preprocessed_data_dropdown.value,
    "selected_full_paths_in_preprocessed_data": preprocessed_data_full_paths_select_multiple.value,
    "selected_datasets_after_splitting_in_train_validation_splits": datasets_to_keep_after_splitting_in_train_validation_splits_select_multiple.value,
    "selected_datasets_after_splitting_in_test_split": datasets_to_keep_after_splitting_in_test_split_select_multiple.value,
    "rng_seed": rng_seed_bounded_int_text.value,
    "shuffle_data_when_splitting": shuffle_data_when_splitting_checkbox.value,
    "train_split_ratio": train_split_ratio_bounded_float_text.value,
    "validation_split_ratio": validation_split_ratio_bounded_float_text.value,
    "test_split_ratio": test_split_ratio_bounded_float_text.value,
    **json.load(input_file)
  }, output_file, sort_keys=True, indent=4)